In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc

---------

In [3]:
from nn_rag import Knowledge

In [4]:
# create instance of capability
kn = Knowledge.from_memory()

In [5]:
tbl = kn.set_source_uri('hadron/source/Gen AI Best Practices.pdf').load_source_canonical()

In [16]:
import io
import requests
import fitz
import pymupdf4llm

def _get_text_from_doc(doc, as_pages, as_markdown) -> list:
    if as_markdown:
        if as_pages
        texts = []
        table = []
        for chunk in pymupdf4llm.to_markdown(doc, page_chunks=True):
            text.append(chunk['text'])
            table.append
        else:
            return pymupdf4llm.to_markdown(doc, page_chunks=as_pages)
    elif as_pages:
        texts = []
        for page in doc:  # iterate the document pages
            texts.append(page.get_text().encode().decode())
        return texts
    return [chr(12).join([page.get_text() for page in doc]).encode().decode()]

filename = 'hadron/source/Gen AI Best Practices.pdf'
# filename = "https://assets.circle.so/kvx4ix1f5ctctk55daheobna46hf"
file_type = 'pdf'
markdown = True
pages = True

# request = requests.get(filename)
# filestream = io.BytesIO(request.content)

with fitz.open(filename, filetype=file_type) as doc:
    texts = _get_text_from_doc(doc, pages, markdown)

In [ ]:
texts

In [8]:
def compare_elements(input_list):
    for i in range(len(input_list) - 1):
        current_element = input_list[i]
        next_element = input_list[i + 1]
        
        if current_element < next_element:
            print(f"{current_element} is less than {next_element}")
        elif current_element == next_element:
            print(f"{current_element} is equal to {next_element}")
        else:
            print(f"{current_element} is greater than {next_element}")

# Example usage
example_list = [3, 5, 2, 8, 6]
compare_elements(example_list)


3 is less than 5
5 is greater than 2
2 is less than 8
8 is greater than 6


In [9]:
# dir(spacy.symbols)

In [10]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")
type(nlp)
for text in p['text']:
    doc = nlp(text.as_py())

    chunks = list(doc.noun_chunks)
    
    words = [token.text for token in doc 
             if not token.is_stop and 
                not token.is_punct and 
                not token.is_space and
                not token.text in ['●']]
    
    # noun tokens that arent stop words or punctuations
    nouns = [token.text
             for token in doc
             if (not token.is_stop and
                 not token.is_punct and
                 not token.is_space and
                 not token.text in ['●'] and
                 token.pos_ == "NOUN")]
    
    # five most common tokens
    common_words = dict(Counter(words).most_common(5))
    type(common_words)
    {k: c for k, c in common_words.items() if c >= 3}
    # print(f"words - {[{k: c} for k, c in common_words.items() if c >= 0]}")
    
    # five most common noun tokens
    common_nouns = dict(Counter(nouns).most_common(5))
    [{k: c for k, c in common_nouns.items() if c >= 3}]
    # print(f"nouns - {[{k: c} for k, c in common_nouns.items() if c >= 0]}")


spacy.lang.en.English

NameError: name 'p' is not defined